In [1]:
import torch
from MLP import MLP

In [2]:
# Load in the model
model = MLP()
model.load_state_dict(torch.load("model.pt"))
model.eval()

MLP(
  (layers): Sequential(
    (0): Linear(in_features=24, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=256, out_features=64, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=64, out_features=1, bias=True)
    (16): Sigmoid()
  )
)

In [3]:
# Load in the data
data = torch.load("../data/data_tens.pt").to(torch.float32)

In [4]:
data_X = data[:, :-1]
data_y = data[:, -1].unsqueeze(-1)

In [5]:
data_X

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 1., 0.]])

In [6]:
data_y

tensor([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [7]:
# One hot mapping
def one_hot(map_, num_cls, v):
    if type(v) != list:
        v = [v]
    v = torch.tensor([map_[i] for i in v])
    v_ = torch.zeros((v.shape[0], num_cls + 1))
    v_[torch.arange(v.shape[0]), v] = 1
    return v_

# 
def vars_to_vec(Age, Income, Gender, Married, Education, Past_Accidents, Has_Children, Has_Sports_Car, Vehicle_Year):
    # Age
    if Age > 0:
        if Age <= 25:
            Age = "16-25"
        elif Age <= 39:
            Age = "26-39"
        elif Age <= 64:
            Age = "40-64"
        else:
            Age = "65+"
    map_ = {"16-25":0, "26-39":1, "40-64":2, "65+":3, "none":4}
    Age_oh = one_hot(map_, 4, Age)
    Age_oh[Age_oh[:, -1] == 1] *= 0
    Age = Age_oh[:, :-1]
            
    # Income
    if Income >= 0:
        if Income < 35_800:
            Income = "poverty"
        elif Income < 52_200:
            Income = "working class"
        elif Income < 156_600:
            Income = "middle class"
        else:
            Income = "upper class"
    else:
        Income = "none"
    map_ = {"poverty":0, "working class":1, "middle class":2, "upper class":3, "none":4}
    Income_oh = one_hot(map_, 4, Income)
    Income_oh[Income_oh[:, -1] == 1] *= 0
    Income = Income_oh[:, :-1]
    
    # Gender
    map_ = {"female":0, "male":1, "NA":2}
    Gender_oh = one_hot(map_, 2, Gender)
    Gender_oh[Gender_oh[:, -1] == 1] *= 0
    Gender = Gender_oh[:, :-1]
    
    # Married
    map_ = {"False":0, "True":1, "NA":2}
    Married_oh = one_hot(map_, 2, Married)
    Married_oh[Married_oh[:, -1] == 1] *= 0
    Married = Married_oh[:, :-1]
    
    # Education
    map_ = {"NA":0, "HS":1, "B":2, "M":3, "PHD":4}
    Education = one_hot(map_, 4, Education)
    
    # Past_Accidents
    Past_Accidents = torch.tensor([[Past_Accidents]])/15
    
    # Has_Children
    map_ = {"False":0, "True":1, "NA":2}
    Has_Children_oh = one_hot(map_, 2, Has_Children)
    Has_Children_oh[Has_Children_oh[:, -1] == 1] *= 0
    Has_Children = Has_Children_oh[:, :-1]
    
    # Has_Sports_Car
    map_ = {"False":0, "True":1, "NA":2}
    Has_Sports_Car_oh = one_hot(map_, 2, Has_Sports_Car)
    Has_Sports_Car_oh[Has_Sports_Car_oh[:, -1] == 1] *= 0
    Has_Sports_Car = Has_Sports_Car_oh[:, :-1]
    
    # Vehicle_Year
    if Vehicle_Year > 2000:
        if Vehicle_Year < 2015:
            Vehicle_Year = "before 2015"
        else:
            Vehicle_Year = "before 2015"
    else:
        Vehicle_Year = "none"
    map_ = {"before 2015":0, "after 2015":1, "none":2}
    Vehicle_Year_oh = one_hot(map_, 2, Vehicle_Year)
    Vehicle_Year_oh[Vehicle_Year_oh[:, -1] == 1] *= 0
    Vehicle_Year = Vehicle_Year_oh[:, :-1]
    
    
    return torch.cat((Age, Income, Gender, Married, Education, Past_Accidents,
                      Has_Children, Has_Sports_Car, Vehicle_Year), axis=-1)

In [11]:
Age = 50
Income = 100_000_000
Gender = "NA"
Married = "True"
Education = "HS"
Past_Accidents = 0
Has_Children = "True"
Has_Sports_Car = "True"
Vehicle_Year = 2015

In [12]:
vec = vars_to_vec(Age, Income, Gender, Married, Education, Past_Accidents, Has_Children, Has_Sports_Car, Vehicle_Year)

In [13]:
model(vec)

tensor([[0.2456]], grad_fn=<SigmoidBackward0>)